In [1]:
import time
import numpy as np
import pandas as pd
import random
import torch as t
import copy
import matplotlib.pyplot as plt

from fastcore.foundation import patch
from nixtla.data.tsdataset import TimeSeriesDataset
from nixtla.data.tsloader_fast import TimeSeriesLoader as TimeSeriesLoaderFast
from nixtla.data.tsloader_pinche import TimeSeriesLoader as TimeSeriesLoaderPinche
from nixtla.data.tsloader_general import TimeSeriesLoader as TimeSeriesLoaderGeneral

from nixtla.models.tcn.tcn import TCN

# NP

In [2]:
from nixtla.data.datasets.epf import EPF, EPFInfo
Y_df, X_df = EPF.load(directory='data', group=EPFInfo.groups[0])
train_outsample_mask = np.ones(len(Y_df))
train_outsample_mask[-365 * 24:] = 0
sum(train_outsample_mask)
epf_dataset = TimeSeriesDataset(Y_df=Y_df, S_df=None, X_df=X_df, ts_train_mask=train_outsample_mask)
# print('X: time series features, of shape (#series,#times,#features): \t' + str(X.shape))
# print('Y: target series (in X), of shape (#series,#times): \t \t' + str(Y.shape))
# print('S: static features, of shape (#series,#features): \t \t' + str(S.shape))
Y_df.head()

Processing dataframes ...
Creating ts tensor ...


,unique_id,ds,y
0,NP,2013-01-01 00:00:00,31.05
1,NP,2013-01-01 01:00:00,30.47
2,NP,2013-01-01 02:00:00,28.92
3,NP,2013-01-01 03:00:00,27.88
4,NP,2013-01-01 04:00:00,26.96


In [3]:
train_loader = TimeSeriesLoaderGeneral(ts_dataset=epf_dataset,
                                    model='tcn',
                                    offset=24,
                                    window_sampling_limit=365*4*24, 
                                    input_size=7*24,
                                    output_size=24,
                                    idx_to_sample_freq=1,
                                    batch_size=1024,
                                    is_train_loader=True)

val_loader = TimeSeriesLoaderGeneral(ts_dataset=epf_dataset,
                                  model='tcn',
                                  offset=24,
                                  window_sampling_limit=365*4*24, 
                                  input_size=7*24,
                                  output_size=24,
                                  idx_to_sample_freq=1,
                                  batch_size=1024,
                                  is_train_loader=False)

In [4]:
tcn = TCN(input_size_multiplier=7,
          output_size=24,
          n_channels = [10,10,1],
          kernel_size=3,
          initialization='lecun_normal',
          learning_rate=0.001,
          lr_decay=0.5,
          n_lr_decay_steps=3,
          weight_decay=0,
          l1_inputs=0,
          dropout_prob=0.1,
          max_epochs=10,
          early_stopping=100,
          loss='ME',
          frequency='Y',
          random_seed=1,
          seasonality=4,
          device=None)

In [5]:
tcn.fit(train_ts_loader=train_loader, eval_epochs=1)

============================== Start fitting ==============================
Number of exogenous variables: 11
Number of epochs: 10
insample_y.shape torch.Size([1024, 11, 168])


RuntimeError: The size of tensor a (24) must match the size of tensor b (168) at non-singleton dimension 2